In [1]:
%matplotlib inline
# Load all necessary packages
import sys
import numpy as np
import pandas as pd
import sklearn

sys.path.append("../")
from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
                import load_preproc_data_adult, load_preproc_data_compas

from aif360.algorithms.postprocessing import EqOddsPostprocessing

from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

2024-05-09 22:27:36.707066: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
## import dataset
import csv
from aif360.datasets import BinaryLabelDataset
dataset_used = "/Users/a/Downloads/student+performance/student/student-mat.csv"
headers = "school;sex;age;address;famsize;Pstatus;Medu;Fedu;Mjob;Fjob;reason;guardian;traveltime;studytime;failures;schoolsup;famsup;paid;activities;nursery;higher;internet;romantic;famrel;freetime;goout;Dalc;Walc;health;absences;G1;G2;G3"
separated_headers = headers.split(';')
df = pd.read_csv(dataset_used, delimiter=";")
d_g3 = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:1, 12:1, 13:1, 14:1, 15:1, 16:1, 17:1, 18:1, 19:1, 20:1} #convert into a binary outcome
df['G3'] = df['G3'].map(d_g3)
df['labels'] = df['G3']
df = df.drop('G3', axis = 'columns')
 
#convert categorical to numeric
d_school = {'GP': 0, 'MS': 1}
d_sex = {'M': 0, 'F': 1}
d_age = {15:0, 16:1, 17:2, 18:3, 19:4, 20:5, 21:6, 22:7}
d_address = {'U': 0, 'R': 1}
d_famsize = {'LE3': 0, 'GT3': 0}
d_pstatus = {'T': 0, 'A': 1}
 
d_mjob = {'teacher': 0, 'health': 1, 'services': 2, 'at_home': 3, 'other': 4}
d_fjob = {'teacher': 0, 'health': 1, 'services': 2, 'at_home': 3, 'other': 4}
d_reason = {'home': 0, 'reputation': 1, 'course': 2, 'other': 3}
d_guardian = {'mother': 0, 'father': 1, 'other': 2} 

d_schoolsup = {'yes': 0, 'no': 1}
d_famsup = {'yes': 0, 'no': 1}
d_paid = {'yes': 0, 'no': 1}
d_activities = {'yes': 0, 'no': 1}
d_nursery = {'yes': 0, 'no': 1}
d_higher = {'yes': 0, 'no': 1}
d_internet = {'yes': 0, 'no': 1}
d_romantic = {'yes': 0, 'no': 1}
 
#apply mappings
df['school'] = df['school'].map(d_school)
df['sex'] = df['sex'].map(d_sex)
df['age'] = df['age'].map(d_age)
df['address'] = df['address'].map(d_address)
df['famsize'] = df['famsize'].map(d_famsize)
df['Pstatus'] = df['Pstatus'].map(d_pstatus)
 
df['Mjob'] = df['Mjob'].map(d_mjob)
df['Fjob'] = df['Fjob'].map(d_fjob)
df['reason'] = df['reason'].map(d_reason)
df['guardian'] = df['guardian'].map(d_guardian)
 
df['schoolsup'] = df['schoolsup'].map(d_paid)
df['famsup'] = df['famsup'].map(d_paid)
df['paid'] = df['paid'].map(d_paid)
df['activities'] = df['activities'].map(d_paid)
df['nursery'] = df['nursery'].map(d_paid)
df['higher'] = df['higher'].map(d_paid)
df['internet'] = df['internet'].map(d_paid)
df['romantic'] = df['romantic'].map(d_paid)
 
df = df.dropna()
df = df.apply(pd.to_numeric)
df.head()

new_data = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=df, label_names = ["labels"], protected_attribute_names=["sex","romantic"])
new_data

               instance weights features                                   \
                                         protected attribute                
                                  school                 sex  age address   
instance names                                                              
0                           1.0      0.0                 1.0  3.0     0.0   
1                           1.0      0.0                 1.0  2.0     0.0   
2                           1.0      0.0                 1.0  0.0     0.0   
3                           1.0      0.0                 1.0  0.0     0.0   
4                           1.0      0.0                 1.0  1.0     0.0   
...                         ...      ...                 ...  ...     ...   
390                         1.0      1.0                 0.0  5.0     0.0   
391                         1.0      1.0                 0.0  2.0     0.0   
392                         1.0      1.0                 0.0  6.0     1.0   

In [24]:
from sklearn.model_selection import train_test_split
dataset_orig = new_data
dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.6], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

In [25]:
# print out some labels, names, etc.
display(Markdown("#### Dataset shape"))
# This line uses the display function from IPython's display module and Markdown function to format the string 
#"#### Dataset shape" as a markdown header. This will show up as a heading in the output cell of a Jupyter notebook.
print(dataset_orig_train.features.shape)
#The shape is a tuple that indicates the number of instances and features in the dataset (rows, columns).
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
#In the context of fairness, favorable labels might denote a positive outcome (like being granted bail),
#while unfavorable labels might denote a negative outcome (like being denied bail)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
#This prints the names of the protected attributes. Protected attributes are typically those that 
#should not be used for decision-making due to fairness considerations (e.g., race, gender).
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, dataset_orig_train.unprivileged_protected_attributes)
#This prints the values within the protected attributes that are considered privileged and unprivileged. 
#Privileged values are those that traditionally benefit from societal bias (e.g., being male in a gender attribute), 
#while unprivileged values are those that traditionally do not (e.g., being female in a gender attribute).
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names) #features that are used to train models

#### Dataset shape

(237, 32)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sex', 'romantic']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']


In [26]:
unprivileged_groups = [{"sex" : 0}]
privileged_groups = [{"sex" : 1}]

In [27]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
#BinaryLabelDatasetMetric is a class that provides metrics to evaluate how different the outcomes are across groups defined by protected attributes. 
#It is being used here to calculate the mean difference in outcomes.
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

metric_orig_valid = BinaryLabelDatasetMetric(dataset_orig_valid, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original validation dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_valid.mean_difference())

metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())
# the unprivileged group received less favorable outcomes compared to the privileged group.

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.086490


#### Original validation dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.012179


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.148649


In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve

# Placeholder for predicted and transformed datasets
dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)

dataset_new_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_new_test_pred = dataset_orig_test.copy(deepcopy=True)

# Logistic regression classifier and predictions for training data
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()
lmod = LogisticRegression()
lmod.fit(X_train, y_train)

fav_idx = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]
y_train_pred_prob = lmod.predict_proba(X_train)[:,fav_idx]

# Prediction probs for validation and testing data
X_valid = scale_orig.transform(dataset_orig_valid.features)
y_valid_pred_prob = lmod.predict_proba(X_valid)[:,fav_idx]

X_test = scale_orig.transform(dataset_orig_test.features)
y_test_pred_prob = lmod.predict_proba(X_test)[:,fav_idx]

class_thresh = 0.5
dataset_orig_train_pred.scores = y_train_pred_prob.reshape(-1,1)
dataset_orig_valid_pred.scores = y_valid_pred_prob.reshape(-1,1)
dataset_orig_test_pred.scores = y_test_pred_prob.reshape(-1,1)

y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
y_train_pred[y_train_pred_prob >= class_thresh] = dataset_orig_train_pred.favorable_label
y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_orig_train_pred.unfavorable_label
dataset_orig_train_pred.labels = y_train_pred

y_valid_pred = np.zeros_like(dataset_orig_valid_pred.labels)
y_valid_pred[y_valid_pred_prob >= class_thresh] = dataset_orig_valid_pred.favorable_label
y_valid_pred[~(y_valid_pred_prob >= class_thresh)] = dataset_orig_valid_pred.unfavorable_label
dataset_orig_valid_pred.labels = y_valid_pred
    
y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig_test_pred.favorable_label
y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig_test_pred.unfavorable_label
dataset_orig_test_pred.labels = y_test_pred

In [29]:
cm_pred_train = ClassificationMetric(dataset_orig_train, dataset_orig_train_pred,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original-Predicted training dataset"))
print("Difference in GFPR between unprivileged and privileged groups")
print(cm_pred_train.difference(cm_pred_train.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(cm_pred_train.difference(cm_pred_train.generalized_false_negative_rate))

cm_pred_valid = ClassificationMetric(dataset_orig_valid, dataset_orig_valid_pred,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original-Predicted validation dataset"))
print("Difference in GFPR between unprivileged and privileged groups")
print(cm_pred_valid.difference(cm_pred_valid.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(cm_pred_valid.difference(cm_pred_valid.generalized_false_negative_rate))

cm_pred_test = ClassificationMetric(dataset_orig_test, dataset_orig_test_pred,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Original-Predicted testing dataset"))
print("Difference in GFPR between unprivileged and privileged groups")
print(cm_pred_test.difference(cm_pred_test.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(cm_pred_test.difference(cm_pred_test.generalized_false_negative_rate))

#### Original-Predicted training dataset

Difference in GFPR between unprivileged and privileged groups
-0.014615419416075234
Difference in GFNR between unprivileged and privileged groups
-0.03541346004777881


#### Original-Predicted validation dataset

Difference in GFPR between unprivileged and privileged groups
-0.09086255425690183
Difference in GFNR between unprivileged and privileged groups
-0.09405821419402018


#### Original-Predicted testing dataset

Difference in GFPR between unprivileged and privileged groups
0.2330227705887405
Difference in GFNR between unprivileged and privileged groups
-0.06559023115594534


In [41]:
#Logistic Accuracy before Reweighing
original_model_lr = LogisticRegression(solver='liblinear', random_state=42)
original_model_lr.fit(dataset_orig_train.features, dataset_orig_train.labels.ravel())

LogisticRegression(random_state=42, solver='liblinear')

In [42]:
from sklearn.metrics import accuracy_score
test_pred_lr = original_model_lr.predict(dataset_orig_test.features)

accuracy_before_lr = accuracy_score(dataset_orig_test.labels, test_pred_lr)

print("Logitsic Accuracy_before:", accuracy_before_lr)

Logitsic Accuracy_before: 0.8607594936708861


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np

#Same as Logistic
dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
#standardscaler
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()
#the random forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
#get index of the favorable class in the dataset
fav_idx = np.where(rf_model.classes_ == dataset_orig_train.favorable_label)[0][0]
#predict probabilities for the training, validation, and testing sets
y_train_pred_prob = rf_model.predict_proba(X_train)[:, fav_idx]

X_valid = scale_orig.transform(dataset_orig_valid.features)
y_valid_pred_prob = rf_model.predict_proba(X_valid)[:, fav_idx]

X_test = scale_orig.transform(dataset_orig_test.features)
y_test_pred_prob = rf_model.predict_proba(X_test)[:, fav_idx]

In [7]:
class_thresh = 0.5

# Update prediction score
dataset_orig_train_pred.scores = y_train_pred_prob.reshape(-1, 1)
dataset_orig_valid_pred.scores = y_valid_pred_prob.reshape(-1, 1)
dataset_orig_test_pred.scores = y_test_pred_prob.reshape(-1, 1)

y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
y_train_pred[y_train_pred_prob >= class_thresh] = dataset_orig_train_pred.favorable_label
y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_orig_train_pred.unfavorable_label
dataset_orig_train_pred.labels = y_train_pred

y_valid_pred = np.zeros_like(dataset_orig_valid_pred.labels)
y_valid_pred[y_valid_pred_prob >= class_thresh] = dataset_orig_valid_pred.favorable_label
y_valid_pred[~(y_valid_pred_prob >= class_thresh)] = dataset_orig_valid_pred.unfavorable_label
dataset_orig_valid_pred.labels = y_valid_pred

y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig_test_pred.favorable_label
y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig_test_pred.unfavorable_label
dataset_orig_test_pred.labels = y_test_pred

# Calculate fairness metrics
cm_pred_train = ClassificationMetric(dataset_orig_train, dataset_orig_train_pred,
                                     unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups)
print("#### Original-Predicted training dataset")
print("Difference in GFPR between unprivileged and privileged groups")
print(cm_pred_train.difference(cm_pred_train.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(cm_pred_train.difference(cm_pred_train.generalized_false_negative_rate))

cm_pred_valid = ClassificationMetric(dataset_orig_valid, dataset_orig_valid_pred,
                                     unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups)
print("#### Original-Predicted validation dataset")
print("Difference in GFPR between unprivileged and privileged groups")
print(cm_pred_valid.difference(cm_pred_valid.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(cm_pred_valid.difference(cm_pred_valid.generalized_false_negative_rate))

cm_pred_test = ClassificationMetric(dataset_orig_test, dataset_orig_test_pred,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
print("#### Original-Predicted testing dataset")
print("Difference in GFPR between unprivileged and privileged groups")
print(cm_pred_test.difference(cm_pred_test.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(cm_pred_test.difference(cm_pred_test.generalized_false_negative_rate))

#### Original-Predicted training dataset
Difference in GFPR between unprivileged and privileged groups
0.029992498124531128
Difference in GFNR between unprivileged and privileged groups
-4.595588235294101e-05
#### Original-Predicted validation dataset
Difference in GFPR between unprivileged and privileged groups
-0.0634429824561403
Difference in GFNR between unprivileged and privileged groups
-0.07487012987012989
#### Original-Predicted testing dataset
Difference in GFPR between unprivileged and privileged groups
0.0421052631578947
Difference in GFNR between unprivileged and privileged groups
-0.04464285714285715


In [36]:
original_model_rf = RandomForestClassifier(random_state=42)
original_model_rf.fit(dataset_orig_train.features, dataset_orig_train.labels.ravel())

RandomForestClassifier(random_state=42)

In [43]:
from sklearn.metrics import accuracy_score
test_pred_rf_before = original_model_rf.predict(dataset_orig_test.features)

accuracy_before_rf = accuracy_score(dataset_orig_test.labels, test_pred_rf_before)

print("Random Forest Accuracy before:", accuracy_before_rf)


Random Forest Accuracy before: 0.9113924050632911


In [37]:
## Before this cell, are all models before reweighing
## After this cell, will be the reweighing process 

In [39]:
#Reweighing Process.
from aif360.algorithms.preprocessing import Reweighing

reweighing = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
dataset_orig_train_rw = reweighing.fit_transform(dataset_orig_train)

weights = dataset_orig_train_rw.instance_weights
weights

array([0.91826596, 1.12359782, 1.12359782, 1.07344218, 1.07344218,
       0.9220383 , 1.07344218, 1.07344218, 1.07344218, 1.07344218,
       0.9220383 , 0.91826596, 0.9220383 , 1.07344218, 0.9220383 ,
       1.12359782, 0.91826596, 0.9220383 , 0.9220383 , 0.91826596,
       1.07344218, 1.07344218, 1.07344218, 1.07344218, 0.9220383 ,
       1.07344218, 0.91826596, 0.91826596, 1.07344218, 0.91826596,
       0.9220383 , 1.12359782, 0.9220383 , 0.91826596, 0.91826596,
       0.91826596, 1.07344218, 0.91826596, 0.9220383 , 0.9220383 ,
       0.91826596, 1.07344218, 0.91826596, 0.9220383 , 0.9220383 ,
       1.07344218, 0.91826596, 0.91826596, 0.9220383 , 0.9220383 ,
       1.07344218, 1.07344218, 0.91826596, 0.9220383 , 0.9220383 ,
       0.9220383 , 0.91826596, 0.9220383 , 1.12359782, 1.07344218,
       0.91826596, 1.07344218, 0.91826596, 0.91826596, 1.07344218,
       0.91826596, 1.07344218, 1.12359782, 0.91826596, 0.9220383 ,
       1.07344218, 0.9220383 , 0.9220383 , 0.9220383 , 0.92203

In [45]:
#Logistic After Reweighing
from sklearn.linear_model import LogisticRegression

model_lr_after = LogisticRegression(solver='liblinear', random_state=42)
model_lr_after.fit(dataset_orig_train_rw.features, dataset_orig_train_rw.labels.ravel(), sample_weight=dataset_orig_train_rw.instance_weights)
model_lr_after

LogisticRegression(random_state=42, solver='liblinear')

In [51]:
from sklearn.metrics import accuracy_score

test_pred_lr_after = model_lr_after.predict(dataset_orig_test.features)

accuracy_lr_after = accuracy_score(dataset_orig_test.labels, test_pred_lr_after)
print("Logistic Regression Accuracy After Reweighing:", accuracy_lr_after)

testing_metrics = BinaryLabelDatasetMetric(dataset_orig_test, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print("Mean difference (testing set):", testing_metrics.mean_difference())

Logistic Regression Accuracy After Reweighing: 0.8734177215189873
Mean difference (testing set): 0.14864864864864863


In [13]:
#Random Forest after reweighing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [14]:
dataset_orig_train.labels

array([[1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],

In [47]:
model_after_rf = RandomForestClassifier(random_state=42)
model_after_rf.fit(dataset_orig_train_rw.features, dataset_orig_train_rw.labels.ravel(), sample_weight=dataset_orig_train_rw.instance_weights)

RandomForestClassifier(random_state=42)

In [48]:
test_pred_rf_after = model_after_rf.predict(dataset_orig_test.features)

accuracy_rf_after = accuracy_score(dataset_orig_test.labels, test_pred_rf_after)
print("Accuracy:", accuracy_rf_after)

testing_metrics = BinaryLabelDatasetMetric(dataset_orig_test, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
print("Mean difference (testing set):", testing_metrics.mean_difference())

Accuracy: 0.9113924050632911
Mean difference (testing set): 0.14864864864864863
